In [1]:
import gradio as gr
import pickle
import gzip
import numpy as np
import pandas as pd


with gzip.open("new_pickle/model.pklz", "rb") as f:
    w2v_model = pickle.load(f)
    tfidf_vectorizer = pickle.load(f)
    rfc = pickle.load(f)


movie_data = pd.read_csv("finalmovie.csv")

def predict_genre(synopsis, num_titles):
    # Process user input
    user_input_processed = tfidf_vectorizer.transform([synopsis])
    user_input_vector = np.zeros(w2v_model.vector_size)
    for word in synopsis.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            user_input_vector += word_vector_tfidf
    user_input_vector /= len(synopsis.split())

    # Predict the genre
    genre_prediction = rfc.predict([user_input_vector])[0]
    
    # Filter the movie titles based on the predicted genre
    movie_titles = movie_data[movie_data['new_genre'] == genre_prediction]['title'].tolist()
    
    # Limit the number of movie titles displayed based on the user input
    if len(movie_titles) > num_titles:
        movie_titles = movie_titles[:num_titles]
    
    # Format the movie titles as a bulleted list
    movie_titles_bullet = "\n".join([f"- {title}" for title in movie_titles])
    
    # Return the predicted genre and movie titles
    return genre_prediction, movie_titles_bullet


# Create the Gradio interface with a Slider for the number of suggested movie titles
synopsis_input = gr.Textbox(label="Enter the synopsis of the movie")
num_titles_input = gr.Slider(minimum=5, maximum=20, step=5, label="Number of suggested movie titles")
genre_output = gr.Textbox(label="Predicted genre")
titles_output = gr.Textbox(label="Suggested movie titles with the predicted genre")
gr.Interface(fn=predict_genre, inputs=[synopsis_input, num_titles_input], outputs=[genre_output, titles_output], 
             title="GenreOracle (v1.0-beta1)",
             description="Enter the synopsis of a movie and get suggestions for similar movies").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
